In [8]:
import torch
import torchvision.transforms as transforms
import lightning as L
import torchmetrics

import brain_tumor_dataset as btd

In [5]:
# transform the images to ensure equal size and tensors
transform = transforms.Compose([
	transforms.Resize((512, 512)),
	transforms.ToTensor(),
	transforms.Grayscale()	
])

train_dataset = btd.BrainTumorDataset(btd.TRAIN_DATA_PATH, transform=transform)

# train_dataset = BrainTumorDataset(train_df, train_dir, transform=transform)
test_dataset = btd.BrainTumorDataset(btd.TEST_DATA_PATH, transform=transform)

batch_size = 32

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Display the images in a grid from one batch in the DataLoader
# for imgs, labels in train_loader:
#     btd.show_image_grid(imgs, labels, rows = batch_size//4, cols = 4)
#     break  

In [ ]:
class VisionTransformer(L.LightningModule):
	def __init__(self, 
                 embedding_dim=64, 
                 patch_size=32, 
                 num_layers=4, 
                 n_heads=8, 
                 dim_feedforward=64,
				learning_rate=1e-4):
		super().__init__()

		# hyperparameters
		self.learning_rate = learning_rate
		self.criterion = torch.nn.CrossEntropyLoss()

		# metrics
		self.train_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=4)
		self.val_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=4)

		self.embedding = torch.nn.Conv2d(
			1, embedding_dim, kernel_size=patch_size, stride=patch_size
		)

		# class token for classification
		self.class_token = torch.nn.Parameter(torch.randn(1, 1, embedding_dim))
		# positional embedding
		self.positional_embedding = torch.nn.Parameter(torch.randn(1, 1, embedding_dim))
		# transformer encoder layer
		self.encoder = torch.nn.TransformerEncoder( # we can add a norm between each encoder layer here
			torch.nn.TransformerEncoderLayer( 
				d_model=embedding_dim, nhead=n_heads, dim_feedforward=dim_feedforward
			),
			num_layers=num_layers,
			enable_nested_tensor=False,
		)

		# linear layer for classification
		# TODO: add a linear layer for classification	

	def forward(self, x):
		x = self.embedding(x)
		# TODO
		return x
    
	def training_step(self, batch, batch_idx):
		inputs, labels = batch
		output = self(inputs)
		loss = self.criterion(output, labels)

		self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
		self.train_accuracy(output, labels)
		self.log('train_accuracy', self.train_accuracy, on_step=True, on_epoch=True)

		return loss

	def validation(self, batch, batch_idx):
		pass

	def configure_optimizers(self):
		return torch.optim.Adam(self.parameters(), lr=self.learning_rate)


# test with random image
model = VisionTransformer(embedding_dim=8)
imgs, labels = next(iter(train_loader))
output = model(imgs[0])
output = output.flatten(start_dim=1)
print(output.shape)

torch.Size([8, 256])


In [7]:
trainer = L.Trainer(max_epochs=10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Andreas\anaconda3\envs\deep_learning\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
